# Game 3 - Hospital Resources

Skillfully allocating medical resources during a pandemic can save lives. Unexpected and extreme medical events not only disrupt hospitals' normal routines and protocol, but strain resources such as trained staff, space and pharmaceuticals, in addition to PPE and life-saving medical equipment. Determining how to most effectively allocate these resources is paramount to assuring the wellbeing of Sailors far from port.

In this scenario, you will oversee the medical bay of the USS GHOST where COVID-19 has taken hold of the crew. Your task is to create an algorithm to deploy the ship's available resources in a manner that will ensure as many Sailors as possible recover from the virus.

## brainstorming:
- reinforcement learning?

In [3]:
import pandas as pd
import numpy as np


* +30 10 ventilators (reusable)
* +20 10 oxygen masks (reusable)
    * cannot be combined with ventilator
* +15 10 plasma
* +30 7 remdesivir
* +25 20 Dexamethasone
* +15 10 casirivimab
* +10 17 chloroquine
* 30 beds
    * bed boosts health 5 points
    * bed allocation required for other treatments
* final health score = final health + bed bonus + rate of decline + treatment bonuses
* penalty of -100 if any sailor final health is 0


In [21]:
sailors_df = pd.read_csv("day1data.csv", header=1, index_col=0)
sailors_df.rename(columns={"Health":"hp_start", "Rate of health decline": "hp_decline_rate"}, inplace=True)
columns = ["resource", "count", "amt_used", "reusable", "health_bonus"]
data = np.array([["bed", 30, 0, True, 5],
          ["ventilator", 10, 0, True, 30],
          ["oxygen mask", 10, 0, True, 20],
          ["plasma", 10, 0, False, 15],
          ["remdesivir", 7, 0, False, 30],
          ["dexamethasone", 20, 0, False, 25],
          ["casirivimab", 10, 0, False, 15],
          ["chloroquine", 17, 0, False, 10]])
resources_df = pd.DataFrame(data=data, columns=columns)
print(resources_df,"\n")
print(sailors_df)

        resource count amt_used reusable health_bonus
0            bed    30        0     True            5
1     ventilator    10        0     True           30
2    oxygen mask    10        0     True           20
3         plasma    10        0    False           15
4     remdesivir     7        0    False           30
5  dexamethasone    20        0    False           25
6    casirivimab    10        0    False           15
7    chloroquine    17        0    False           10 

    hp_start  hp_decline_rate
ID                           
1        100               -1
2         75               -1
3         65               -7
4         55              -25
5         33              -30
6         89              -50
7         90              -60
8         45              -65
9        100              -79
10        95              -88
11        60              -62
12        72              -18
13        55              -68
14        28              -67
15        39              -89
16

In [88]:
def allocation(sailors_df, resources_df):
    sailors_df = sailors_df.copy()
    sailors_df["hp_final"] = sailors_df["hp_start"]+sailors_df["hp_decline_rate"]
    sailors_df.sort_values(by=["hp_final"], inplace=True)
    
    add_beds = sailors_df.iloc[:30].index
    sailors_df.loc[add_beds, "bed"] = True
    sailors_df.loc[sailors_df["bed"]==True, "hp_final"] += 5
    
    add_ventilators = sailors_df.iloc[:10].index
    sailors_df.loc[add_ventilators, "ventilator"] = True
    sailors_df.loc[sailors_df["ventilator"]==True, "hp_final"] += 30
    
    oxymask_candidates = sailors_df[(sailors_df["ventilator"]!=True) & (sailors_df["bed"]==True)]
    add_oxymask = oxymask_candidates.iloc[:10].index
    sailors_df.loc[add_oxymask, "oxygen mask"] = True
    sailors_df.loc[sailors_df["oxygen mask"]==True, "hp_final"] += 20
    
    sailors_df["100_diff"] = 100 - sailors_df["hp_final"]
    
    return sailors_df

def calc_final_score(allocation_df):
    final_health = allocation_df["hp_final"].apply(lambda x: 100 if x > 100 else x)
    final_health = final_health.apply(lambda x: -100 if x < 1 else x)
    return final_health.sum()
naive_allocation(sailors_df, resources_df)

,hp_start,hp_decline_rate,hp_final,bed,ventilator,oxygen mask,100_diff
ID,,,,,,,
20,20,-80,-25,True,True,NaN,125
15,39,-89,-15,True,True,NaN,115
14,28,-67,-4,True,True,NaN,104
8,45,-65,15,True,True,NaN,85
13,55,-68,22,True,True,NaN,78
11,60,-62,33,True,True,NaN,67
5,33,-30,38,True,True,NaN,62
10,95,-88,42,True,True,NaN,58
9,100,-79,56,True,True,NaN,44
